In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.color import rgb2gray
from skimage import feature, filters

from skimage.transform import hough_line, hough_line_peaks
from matplotlib import cm


from skimage.morphology import dilation, closing,white_tophat, disk
from skimage import measure
%matplotlib widget

In [2]:
import glob
class0 = glob.glob('patches/landscape-tree-water-forest-grass-road-1411714-pxhere.com.jpg/*')
class1 = glob.glob('patches/ryan-searle-1jckkOd-GU8-unsplash.jpg/*')
x_0 = np.array([np.asarray(Image.open(fname)) for fname in class0])
x_1 = np.array([np.asarray(Image.open(fname)) for fname in class1])

In [3]:
x_0.shape

(300, 256, 256, 3)

In [4]:
x_1.shape

(413, 256, 256, 3)

In [5]:
saved_data=np.load("runs/2021-05-04 13:54:45.752992/discrete_histories.npz")
saved_frame = saved_data["frames"][:,:256,:,:]

In [6]:
saved_frame.shape

(1000, 256, 256, 3)

In [7]:
stacked=np.vstack((x_0, x_1))
labels=np.array([0]*x_0.shape[0]+[1]*x_1.shape[0])
labels.shape

(713,)

In [8]:
from numpy.random import default_rng
from PIL import Image
from numpy import asarray

rng = default_rng()
n=200
numbers = rng.choice(stacked.shape[0], size=n, replace=False)
numbers_open = rng.choice(saved_frame.shape[0], size=n, replace=False)


In [9]:
real_imgs=[stacked[numbers], labels[numbers]]

In [10]:
def seg_sim(image):
    aux2=np.zeros((image.shape[0],256,256,1), dtype=np.uint8)
    aux2[np.where(image[:,:256, :, 1] == 105)]= [255]
    aux2[np.where(image[:,:256, :, 1] == 107)]= [255]
    aux2[np.where(image[:,:256, :, 1] == 102)]= [255]
    return aux2

In [11]:
def do_canny(imgs,img_type):
   
    if img_type == 'real':
        labels= imgs[1]
        imgs=imgs[0]
        img_gray = rgb2gray(imgs)
        for i,_ in enumerate(img_gray):
            if labels[i]==1:
                #brown
                img_gray[i][img_gray[i]<=0.5]=0
                img_gray[i][img_gray[i]>0.5]=1
            else:
                #grey
                img_gray[i] = feature.canny(img_gray[i], sigma=6, low_threshold=0.1)
    
        edges = img_gray # pointer magic
    else:
        img_gray = np.squeeze(seg_sim(imgs))
        edges=np.array([feature.canny(img, sigma=3, low_threshold=0.2) for img in img_gray])
    return edges

In [12]:
def do_canny_single(img, label,img_type):
    if img_type == 'real':
        img_gray = rgb2gray(img)
        if label==1:
            #brown
            img_gray[img_gray<=0.5]=0
            img_gray[img_gray>0.5]=1
        else:
            #grey
            img_gray = feature.canny(img_gray, sigma=6, low_threshold=0.1)
    
        edges = img_gray # pointer magic
    else:
        img_gray = np.squeeze(seg_sim(img[None,:,:,:]))
        edges=feature.canny(img_gray, sigma=3, low_threshold=0.2)
    return edges

In [13]:
mask_width=20
mask_1=np.zeros(shape=(256, 256))
mask_1[0:mask_width,:]=1

mask_2=np.zeros(shape=(256, 256))
mask_2[:,-mask_width:]=1

mask_3=np.zeros(shape=(256, 256))
mask_3[-mask_width:,:]=1

mask_4=np.zeros(shape=(256, 256))
mask_4[:,0:mask_width]=1
masks=[mask_1,mask_2,mask_3,mask_4]

In [14]:
def test_direction(edges, direction, masks=masks):
    filtered_edges=[]
    titles=[]
    mask = masks[direction-1]
    for edge in edges:
        if direction ==1 or direction ==3:
            filtered=np.clip(filters.sobel_h(edge, mask),0,1)
        else:
            filtered=np.clip(filters.sobel_v(edge, mask),0,1)
        filtered_edges.append(filtered)
        titles.append(str(np.sum(filtered)))
    return np.array(filtered_edges), titles

In [15]:
def plot_images(img,fig,j,n,titles=None,rows=2):
    for i in range(1,n+1):
        fig.add_subplot(rows, n, j*n+i)
        plt.imshow(img[i-1])
        if titles is not None:
            plt.title(titles[i-1])
        plt.axis('off')

In [16]:
edges=do_canny(real_imgs,"real")
fig=plt.figure(figsize=(50, 15))

plot_images(real_imgs[0], fig,0,n)
plot_images(edges, fig, 1, n)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:

def h_trasn(image,label,number):
    # Classic straight-line Hough transform
    
    edge = do_canny_single(image,label,"real")
    # Set a precision of 0.5 degree.
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 720, endpoint=False)
    h, theta, d = hough_line(edge, theta=tested_angles)

    # Generating figure 1
    fig, axes = plt.subplots(1, 5, figsize=(25, 6))
    fig.suptitle(f"Image {number}")
    ax = axes.ravel()

    ax[0].imshow(image, cmap=cm.gray)
    ax[0].set_title('Input image')
    ax[0].set_axis_off()
    
    
    ax[1].imshow(edge, cmap=cm.gray)
    ax[1].set_title('Canny edge detection')
    ax[1].set_axis_off()



    ax[2].imshow(edge, cmap=cm.gray)
    ax[2].set_ylim((edge.shape[0], 0))
    ax[2].set_xlim((0, edge.shape[1]))
    ax[2].set_axis_off()
    ax[2].set_title('Detected lines')
    hspace, angles, dists = hough_line_peaks(h, theta, d)
#     indexes = np.where(hspace > 0) # segment length filter
#     hspace = hspace[indexes]
#     angles = angles[indexes]
#     dists =  dists[indexes]
    
    for _, angle, dist in zip(hspace, angles, dists):
        (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
        ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))
        
    if angles.size !=0:
        x_intersections=[]
        y_intersections=[]
        for i in range(angles.shape[0]):
            for j in range(angles.shape[0]):
                a1 =angles[i]
                d1 =dists[i]
                a2 =angles[j]
                d2 = dists[j]
                if (a1 % np.pi) - (a2 % np.pi) !=0:
                    m1 = np.tan(a1 + np.pi/2)
                    (x1, y1) = d1 * np.array([np.cos(a1), np.sin(a1)])
                    m2 = np.tan(a2 + np.pi/2)
                    (x2, y2) = d2 * np.array([np.cos(a2), np.sin(a2)])
                    x_int = ((m2*x2 - m1*x1) - (y2-y1))/(m2-m1)
                    y_int = m1*(x_int-x1)+y1
                    if x_int > 0 and x_int<256 and y_int > 0 and y_int<256: 
                        x_intersections.append(x_int)
                        y_intersections.append(y_int)
        ax[2].scatter(x_intersections,y_intersections, color='r')
        
        if len(x_intersections) >0 :
            
            mymodel_intersec = np.poly1d(np.polyfit(x_intersections, y_intersections, 2))

            myline = np.linspace(0, 256, 100)

            ax[2].plot(myline,mymodel_intersec(myline),'--r')

    
    
 
        
    closed = closing(edge, disk(20))
    ax[3].imshow(closed, cmap=cm.gray)
    ax[3].set_title('closing')
    ax[3].set_axis_off()
    
    wt= white_tophat(closed, disk(1))
    
    
    
    
    x,y=np.where(wt >0.5)
    if x.size !=0:
        mymodel = np.poly1d(np.polyfit(x, y, 2))

        myline = np.linspace(0, 256, 100)

        ax[4].imshow(wt, cmap=cm.gray)
        ax[4].set_title('white tophat')
        ax[4].set_axis_off()
        ax[4].plot(mymodel(myline),myline)
        ax[4].set_ylim((edge.shape[0], 0))
        ax[4].set_xlim((0, edge.shape[1]))
    
    plt.tight_layout()
    plt.show()
    
    
    tresh=10
    bottom= np.sum(edge[-10:,:])
    print("hspace",hspace)
    print("angles",np.rad2deg(angles))
    print("dists", dists)
    if angles.size !=0 and bottom > 0:
        mean_angle = np.rad2deg(np.mean(angles))
        print("mean angle",mean_angle)
        if mean_angle < - tresh:
            print("left curve")
        elif mean_angle > tresh:
            print("right curve")
        elif mean_angle >= -tresh and mean_angle <= tresh:
            print("Straight road")
        else:
            print("Undefined")
    else:
        print("Undefined")
    
    print("Fitted curve method")
    if bottom > 0:
        d2 =np.polyder(mymodel,m=2)
        d2 = d2(0)*1000
        if d2 < -0.5 :
            print("left curve")
        elif d2> 0.5:
            print("right curve")
        else:
            print("Straight road")
        
        print("d2", d2)
    else:
        print("Undefined")

In [18]:
i =4
h_trasn(stacked[numbers][i], labels[numbers][i],i)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hspace [135 133]
angles [12.75 10.  ]
dists [127.67586207 173.73931034]
mean angle 11.375000000000005
right curve
Fitted curve method
right curve
d2 2.8825172749799663


In [19]:
#for i, (img,label) in enumerate(zip(stacked[numbers], labels[numbers])):
#    h_trasn(img,label,i)

In [20]:

def h_trasn_simple(image,label,number, img_type="real"):
    # Classic straight-line Hough transform
    
    edge = do_canny_single(image,label, img_type)

    # Generating figure 1
    fig, axes = plt.subplots(1, 4, figsize=(25, 6))
    
    ax = axes.ravel()

    ax[0].imshow(image, cmap=cm.gray)
    ax[0].set_title('Input image')
    ax[0].set_axis_off()
    
    ax[1].imshow(edge, cmap=cm.gray)
    ax[1].set_title('Canny edge detection')
    ax[1].set_axis_off()
   


    if img_type=="real":
        closed = closing(edge, disk(20))

        wt= white_tophat(closed, disk(1))


        
    else:
        wt = edge
    
    r = dilation(wt, disk(2))
    r = np.pad(r,2)
    contours = measure.find_contours(r, 0.8)

    
    myline = np.linspace(0, 256, 100)
    
    
    ax[2].imshow(r, cmap=cm.gray)
    ax[2].set_title('Prossesed (closing + white_tophat + dilation + padding + contours)')
    ax[2].set_axis_off()
    for contour in contours:
        ax[2].plot(contour[:, 1], contour[:, 0], linewidth=2)
 
    
    ax[3].imshow(np.zeros(edge.shape),cmap=cm.gray)
    ax[3].set_title('Fitted curve and points')
    ax[3].set_axis_off()
    
    road_type ="Undefined"
    
    defined =True
    if len(contours) !=0:
        contours=contours[np.argmax([c.shape[0] for c in contours ])]    
   
        x = np.vstack(contours)[:,0]
        y = np.vstack(contours)[:,1]
   
        ax[3].scatter(y,x)
   
        mymodel = np.poly1d(np.polyfit(x, y, 2))
        line =mymodel(myline)
        
        
        if mymodel(256) > 300 or  mymodel(256) < -50:
            defined = False
            
        ax[3].plot(line,myline, "--r")
    else:
        defined = False
        road_type="No road"
    ax[3].set_ylim((edge.shape[0], 0))
    ax[3].set_xlim((0, edge.shape[1]))
    
    
    

    bottom= np.sum(edge[-20:,:])
    if bottom > 10 and defined:
        d2 =np.polyder(mymodel,m=2)
        d2 = d2(0)*1000
        if d2 < -1 :
            road_type="left curve"
        elif d2> 1:
            road_type="right curve"
        else:
            road_type="Straight road"
        
        road_type=f"{road_type}\nd2: {d2}"
   
        
    
    fig.suptitle(f"Image {number}  {road_type}")
    plt.tight_layout()
    plt.show()


In [21]:
n=66
img = stacked[numbers][n]
label = labels[numbers][n]
h_trasn_simple(img,label,n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
n=60
img = stacked[numbers][n]
label = labels[numbers][n]
h_trasn_simple(img,label,n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
n=11
img = stacked[numbers][n]
label = labels[numbers][n]
print(label)
h_trasn_simple(img,label,n)


1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
n=4
img = stacked[numbers][n]
label = labels[numbers][n]
h_trasn_simple(img,label,n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
n=72
img = stacked[numbers][n]
label = labels[numbers][n]
h_trasn_simple(img,label,n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
n=72
img = stacked[numbers][n]
label = labels[numbers][n]
plt.imshow(img) #h_trasn_simple(img,label,n)

In [27]:
# x_0_filt = x_0[np.array([4,5,6,7,8,11,23,25,29,37,41,48,64,71])]
# for i, (img,label) in enumerate(zip(x_0_filt, np.zeros(x_0_filt.shape[0]))):
#     h_trasn_simple(img,label,i)

In [28]:
# for i, (img,label) in enumerate(zip(stacked[numbers], labels[numbers])):
#     h_trasn_simple(img,label,i)

In [29]:
# for i, (img,label) in enumerate(zip(x_1, np.ones(x_1.shape[0]))):
#     h_trasn_simple(img,label,i)

In [30]:
i=0
img = saved_frame[numbers_open][i]
h_trasn_simple(img,None,i, img_type="OpenAI")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
i=10
img = saved_frame[numbers_open][i]
h_trasn_simple(img,None,i, img_type="OpenAI")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
i=5
img = saved_frame[numbers_open][i]
h_trasn_simple(img,None,i, img_type="OpenAI")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# for i, img in enumerate(saved_frame[numbers_open]):
#     h_trasn_simple(img,None,i, img_type="OpenAI")

In [34]:
def curvature(image,label, img_type="real"):
    """
    returns: 1:left
             2:straight
             3:right
             4:no road
             5:undefined
    """
    # Classic straight-line Hough transform
    
    edge = do_canny_single(image,label, img_type)

    if img_type=="real":
        closed = closing(edge, disk(20))

        wt= white_tophat(closed, disk(1))

    else:
        wt = edge

    

    r = dilation(wt, disk(2))
    r = np.pad(r,2)
    contours = measure.find_contours(r, 0.8)

    
    road_type =5
    defined =True
    if len(contours) !=0:
        contours=contours[np.argmax([c.shape[0] for c in contours ])]    
   
        x = np.vstack(contours)[:,0]
        y = np.vstack(contours)[:,1]
   
   
        mymodel = np.poly1d(np.polyfit(x, y, 2))
        
        
        if mymodel(256) > 300 or  mymodel(256) < -50:
            defined = False
    else:
        defined = False
        road_type=4
        
    bottom= np.sum(edge[-20:,:])
    if bottom > 10 and defined:
        d2 =np.polyder(mymodel,m=2)
        d2 = d2(0)*1000
        if d2 < -1 :
            road_type = 1
        elif d2> 1:
            road_type= 3
        else:
            road_type= 2
    return road_type
    
    

In [36]:
x_0_curv =[curvature(img,label) for img,label in zip(x_0, np.zeros(x_0.shape[0]))]

In [37]:
x_1_curv = [curvature(img,label) for img,label in zip(x_1, np.ones(x_1.shape[0]))]

KeyboardInterrupt: 

In [ ]:
OpenAI_curv = [curvature(img,None, img_type= "OpenAI") for img in saved_frame]

In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 1.0*height,
                '%d' % int(height),
                ha='center', va='bottom')

In [ ]:
fig = plt.figure(figsize=(15,5))
X = np.arange(4)

plt.title(f"Dataset target distribution for: \nStyle 0 dataset {x_0.shape[0]} examples,\nStyle 1 dataset {x_1.shape[0]} examples,\nOpenAI dataset {saved_frame.shape[0]} examples")


x_0_rect=plt.bar(X + 0.00,np.unique(x_0_curv, return_counts=True)[1], color = 'b', width = 0.25)
x_1_rect=plt.bar(X + 0.25,np.unique(x_1_curv, return_counts=True)[1], color = 'g', width = 0.25)
AI_rect=plt.bar(X + 0.5,np.hstack((np.unique(OpenAI_curv, return_counts=True)[1],0)), color = 'orange', width = 0.25)

plt.legend(labels=['style 0', 'style 1', 'OpenAI'])
autolabel(x_0_rect)
autolabel(x_1_rect)
autolabel(AI_rect)

plt.xticks(X, ["left","straight", "right", "no road", "undefined"])
plt.show()


In [ ]:
print("Style 0")
plt.imshow(x_0[0])

In [ ]:
print("Style 1")
plt.imshow(x_1[0])

In [ ]:
print("OpenAI")
plt.imshow(saved_frame[0])

In [ ]:
def h_trasn_simple(image,label,number, img_type, row , ax, fig):
    # Classic straight-line Hough transform
    
    edge = do_canny_single(image,label, img_type)

    # Generating figure 1
    

    ax[row, 0].imshow(image, cmap=cm.gray)
    ax[row, 0].set_title('Input image')
    ax[row, 0].set_axis_off()
    
    ax[row, 1].imshow(edge, cmap=cm.gray)
    ax[row, 1].set_title('Canny edge detection')
    ax[row, 1].set_axis_off()
   


    if img_type=="real":
        closed = closing(edge, disk(20))

        wt= white_tophat(closed, disk(1))


        
    else:
        wt = edge
    
    r = dilation(wt, disk(2))
    r = np.pad(r,2)
    contours = measure.find_contours(r, 0.8)

    
    myline = np.linspace(0, 256, 100)
    
    
    ax[row, 2].imshow(r, cmap=cm.gray)
    ax[row, 2].set_title('Prossesed (closing + white_tophat + dilation + padding + contours)')
    ax[row, 2].set_axis_off()
    for contour in contours:
        ax[row, 2].plot(contour[:, 1], contour[:, 0], linewidth=2)
 
    
    ax[row, 3].imshow(np.zeros(edge.shape),cmap=cm.gray)
    ax[row, 3].set_title('Fitted curve and points')
    ax[row, 3].set_axis_off()
    
    road_type ="Undefined"
    
    defined =True
    if len(contours) !=0:
        contours=contours[np.argmax([c.shape[0] for c in contours ])]    
   
        x = np.vstack(contours)[:,0]
        y = np.vstack(contours)[:,1]
   
        ax[row, 3].scatter(y,x)
   
        mymodel = np.poly1d(np.polyfit(x, y, 2))
        line =mymodel(myline)
        
        
        if mymodel(256) > 300 or  mymodel(256) < -50:
            defined = False
            
        ax[row, 3].plot(line,myline, "--r")
    else:
        defined = False
        road_type="No road"
    ax[row, 3].set_ylim((edge.shape[0], 0))
    ax[row, 3].set_xlim((0, edge.shape[1]))
    
    
    

    bottom= np.sum(edge[-20:,:])
    if bottom > 10 and defined:
        d2 =np.polyder(mymodel,m=2)
        d2 = d2(0)*1000
        if d2 < -1 :
            road_type="left curve"
        elif d2> 1:
            road_type="right curve"
        else:
            road_type="Straight road"
        
        road_type=f"{road_type}\nd2: {d2}"
   
        
    
    fig.suptitle(f"Image {number}  {road_type}")
    return ax,fig

def curve_detectin(openAI_data,real_style_0,real_style_1)
    n=0
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))

    ax,fig= h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0 , ax, fig)

    ax,fig= h_trasn_simple(real_style_0[n],0,n, "real", 1 , ax, fig)

    ax,fig= h_trasn_simple(real_style_1[n],1,n, "real", 2 , ax, fig)
    plt.tight_layout()

    def update(n):
        ax,fig= h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0 , ax, fig)

        ax,fig= h_trasn_simple(real_style_0[n],0,n, "real", 1 , ax, fig)

        ax,fig= h_trasn_simple(real_style_1[n],1,n, "real", 2 , ax, fig)

    
    anim = FuncAnimation(fig, update, frames=frames, interval=400, save_count=sys.maxsize)
    anim.save(f'Slides/curves.gif', dpi=150, writer='imagemagick')
    print(f'Slides/curves.gif done')
    return anim

In [ ]:
def curve_detectin(openAI_data,real_style_0,real_style_1)
    n=0
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))

    ax,fig= h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0 , ax, fig)

    ax,fig= h_trasn_simple(real_style_0[n],0,n, "real", 1 , ax, fig)

    ax,fig= h_trasn_simple(real_style_1[n],1,n, "real", 2 , ax, fig)
    plt.tight_layout()

    def update(n):
        ax,fig= h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0 , ax, fig)

        ax,fig= h_trasn_simple(real_style_0[n],0,n, "real", 1 , ax, fig)

        ax,fig= h_trasn_simple(real_style_1[n],1,n, "real", 2 , ax, fig)

    
    anim = FuncAnimation(fig, update, frames=frames, interval=400, save_count=sys.maxsize)
    anim.save(f'Slides/curves.gif', dpi=150, writer='imagemagick')
    print(f'Slides/curves.gif done')
    return anim

In [ ]:
def curve_detectin(openAI_data,real_style_0,real_style_1):
    def h_trasn_simple(image,label,number, img_type, row ):
        # Classic straight-line Hough transform

        edge = do_canny_single(image,label, img_type)

        # Generating figure 1

        ax[row, 0].imshow(image, cmap=cm.gray)
        ax[row, 0].set_title('Input image')
        ax[row, 0].set_axis_off()

        ax[row, 1].imshow(edge, cmap=cm.gray)
        ax[row, 1].set_title('Canny edge detection')
        ax[row, 1].set_axis_off()



        if img_type=="real":
            closed = closing(edge, disk(20))

            wt= white_tophat(closed, disk(1))



        else:
            wt = edge

        r = dilation(wt, disk(2))
        r = np.pad(r,2)
        contours = measure.find_contours(r, 0.8)


        myline = np.linspace(0, 256, 100)


        ax[row, 2].imshow(r, cmap=cm.gray)
        ax[row, 2].set_title('Prossesed (closing + white_tophat + dilation + padding + contours)')
        ax[row, 2].set_axis_off()
        for contour in contours:
            ax[row, 2].plot(contour[:, 1], contour[:, 0], linewidth=2)


        ax[row, 3].imshow(np.zeros(edge.shape),cmap=cm.gray)
        ax[row, 3].set_title('Fitted curve and points')
        ax[row, 3].set_axis_off()

        road_type ="Undefined"

        defined =True
        if len(contours) !=0:
            contours=contours[np.argmax([c.shape[0] for c in contours ])]    

            x = np.vstack(contours)[:,0]
            y = np.vstack(contours)[:,1]

            ax[row, 3].scatter(y,x)

            mymodel = np.poly1d(np.polyfit(x, y, 2))
            line =mymodel(myline)


            if mymodel(256) > 300 or  mymodel(256) < -50:
                defined = False

            ax[row, 3].plot(line,myline, "--r")
        else:
            defined = False
            road_type="No road"
        ax[row, 3].set_ylim((edge.shape[0], 0))
        ax[row, 3].set_xlim((0, edge.shape[1]))




        bottom= np.sum(edge[-20:,:])
        if bottom > 10 and defined:
            d2 =np.polyder(mymodel,m=2)
            d2 = d2(0)*1000
            if d2 < -1 :
                road_type="left curve"
            elif d2> 1:
                road_type="right curve"
            else:
                road_type="Straight road"

            road_type=f"{road_type}\nd2: {d2}"



        fig.suptitle(f"Image {number}  {road_type}")
   
    n=0
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))

    h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0)

    h_trasn_simple(real_style_0[n],0,n, "real", 1 )

    h_trasn_simple(real_style_1[n],1,n, "real", 2 )
    plt.tight_layout()

    def update(n):
        h_trasn_simpfor i, img in enumerate(saved_frame[numbers_open]):
#     h_trasn_simple(img,None,i, img_type="OpenAI")le(openAI_data[n],None,n, "OpenAI", 0 )

        h_trasn_simple(real_style_0[n],0,n, "real", 1 )

        h_trasn_simple(real_style_1[n],1,n, "real", 2)

    
    anim = FuncAnimation(fig, update, frames=frames, interval=400, save_count=sys.maxsize)
    anim.save(f'Slides/curves.gif', dpi=150, writer='imagemagick')
    print(f'Slides/curves.gif done')
    return anim

In [ ]:
#curve_detectin(openAI_data,real_style_0,real_style_1)

In [ ]:
n=0
fig, ax = plt.subplots(3, 1, figsize=(20, 15))

ax[0]= h_trasn_simple(openAI_data[n],None,n, "OpenAI", 0)

ax[1]= h_trasn_simple(real_style_0[n],0,n, "real", 1)

ax[2]= h_trasn_simple(real_style_1[n],1,n, "real", 2)
plt.tight_layout()

  

In [ ]:
i=0
img = saved_frame[numbers_open][i]
openAI_data = [img]

In [ ]:
h_trasn_simple(img,None,i, img_type="OpenAI")

In [ ]:
n=11
real_style_0 = [stacked[numbers][11]]
h_trasn_simple(img,label,n)


In [ ]:
plt.imshow(stacked[numbers][n])

In [ ]:
n=72
real_style_1 = [stacked[numbers][n]]
plt.imshow(stacked[numbers][n])

In [ ]:
label = labels[numbers][n]
h_trasn_simple(img,label,n)

In [ ]:
experiments, OpenAI_edges, gan_data_0_edges, gan_data_1_edges

In [39]:
experiments = loaded["frames"]


NameError: name 'loaded' is not defined